Reference: https://github.com/minhkhang1795/FakeNews_RNN/blob/master/FakeNews_RNN.ipynb

# Imports

In [ ]:
import sys
import time
from typing import List

import torch
from torch import nn
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split, Subset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import matplotlib.pyplot as plt

# Change system path to base directory.
sys.path.append("..")
from datasets.fake_news_detection import FakeNewsDetectionDataset
from preprocessing.preprocessor import Preprocessor

# Manual set seed for reproducible results
torch.manual_seed(1)

## Read & process dataset

In [ ]:
# Control which linguistic preprocessing steps should run.
preprocessor = Preprocessor(perform_case_folding=True,
                            remove_stop_words=False,
                            remove_punctuation=False,
                            perform_lemmatization=False,
                            perform_stemming=False)

# Generate training and test datasets.
training_dataset = FakeNewsDetectionDataset("../raw_data/fulltrain.csv", transform=preprocessor.process, trim_length=200)
test_dataset = FakeNewsDetectionDataset("../raw_data/balancedtest.csv", transform=preprocessor.process, trim_length=200)

## Prepare data processing pipelines, generate data batch and iterator

In [ ]:
def yield_tokens(dataset: FakeNewsDetectionDataset) -> List[str]:
    for i in range(len(dataset)):
        sentence, _ = dataset[i]
        yield sentence

vocab = build_vocab_from_iterator(yield_tokens(training_dataset), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
text_pipeline = lambda x: vocab(x)

seq_length=200

def pad_sequence(text):
    seq = np.zeros(seq_length, dtype=int)
    seq[-len(text):] = np.array(text_pipeline(text))[0:seq_length]
    return seq

def collate_batch(batch):
    label_list, text_list = [], []
    for text, label in batch:
        label_list.append(label)
        processed_text = torch.tensor(pad_sequence(text), dtype=torch.int64)
        text_list.append(processed_text)
    label_list = torch.tensor(label_list, dtype=torch.int64)
    # Pads inputs vectors to be of same length (seq_length)
    text_list = nn.utils.rnn.pad_sequence(text_list, batch_first=True, padding_value=0)
    # Label list size: [batch_size]
    # Text list size: [batch_size, seq_length]
    return label_list.to(device), text_list.to(device)

## Training

### Model class

In [ ]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers):
        super(TextClassificationModel, self).__init__()

        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                            batch_first=True)
        self.fc = nn.Linear(hidden_dim, 4)

    def init_hidden(self, batch_size):
        # Note that batch_size must be passed in from the training loop
        # to handle the case where the last batch has fewer data.
        # hidden state and cell state size:
        # [num_layers, batch_size, hidden_size]
        weight = next(self.parameters()).data

        return (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))

    def forward(self, text, hidden):
        # Embedding vector size: [batch_size, seq_length, embed_dim]
        embeds = self.embedding(text)
        # LSTM output size: [batch_size, seq_length, num_layers*hidden_size]
        lstm_out, hidden = self.lstm(embeds, hidden)

        # LSTM final state vector size: [batch_size, num_layers*hidden_size]
        final_state = lstm_out[:,-1,:]

        # Linear layer output size: [batch_size, 4]
        output = self.fc(final_state)
        return output, hidden


In [ ]:
# Hyperparameters
vocab_size = len(vocab)
embedding_dim = 84
hidden_dim = 128
n_layers = 1
EPOCHS = 10 # epoch
LR = 0.001  # learning rate
BATCH_SIZE = 64 # batch size for training

model = TextClassificationModel(vocab_size, embedding_dim, hidden_dim, n_layers)
model.to(device)
    
print(model)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

### Define train/eval functions, split training data

In [ ]:
def train(dataloader):
    # initialize hidden state
    h = model.init_hidden(BATCH_SIZE)

    model.train()
    total_acc, total_count, train_loss = 0, 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text) in enumerate(dataloader):
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        model.zero_grad()
        optimizer.zero_grad()

        # get the output from the model
        output, h = model(text, h)

        # calculate the loss and perform backprop
        loss = criterion(output, label)
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*text.size(0)

        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | loss {:8.3f}'
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              train_loss, total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    # initialize hidden state
    val_h = model.init_hidden(BATCH_SIZE)
    model.eval()
    total_acc, total_count = 0, 0
    tp, fp, fn = [0] * 4, [0] * 4, [0] * 4

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            val_h = tuple([each.data for each in val_h])
            output, val_h = model(text, val_h)

            predicted_label = output.argmax(1)
            total_acc += (predicted_label == label).sum().item()
            total_count += label.size(0)

            for lab in range(4):
                tp[lab] += torch.mul(predicted_label == lab, label == lab).sum().item()
                fp[lab] += torch.mul(predicted_label == lab, label != lab).sum().item()
                fn[lab] += torch.mul(predicted_label != lab, label == lab).sum().item()

    f1s = [tp[lab] / (tp[lab] + 0.5 * (fp[lab] + fn[lab])) for lab in range(4)]
    return total_acc/total_count, f1s[0], f1s[1], f1s[2], f1s[3]
    
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
num_train = int(len(training_dataset) * 0.95)
split_train_: Subset[FakeNewsDetectionDataset]
split_valid_: Subset[FakeNewsDetectionDataset]
split_train_, split_valid_ = random_split(training_dataset, [num_train, len(training_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

### Training Loop

In [ ]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val, f1_0, f1_1, f1_2, f1_3 = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid accuracy {:8.3f} '
          '| f1 of class 0 {:8.3f} | f1 of class 1 {:8.3f} | f1 of class 2 {:8.3f} '
          '| f1 of class 3 {:8.3f} | '.format(epoch,
                                              time.time() - epoch_start_time,
                                              accu_val, f1_0, f1_1, f1_2, f1_3))
    print('-' * 59)

In [ ]:
# Save trained model
torch.save(model.state_dict(), '../trained_models/LSTMModel-%sepoch.pt' % EPOCHS)

In [ ]:
# Load saved model to skip training
model.load_state_dict(torch.load('../trained_models/LSTMModel-%sepoch.pt' % EPOCHS))

### Evaluating on test set

In [ ]:
print('Checking the results of test dataset.')
accu_test, f1_0_test, f1_1_test, f1_2_test, f1_3_test = evaluate(test_dataloader)
print('test accuracy {:8.3f} | test f1 of class 0 {:8.3f} | test f1 of class 1 {:8.3f} '
      '| test f1 of class 2 {:8.3f} | test f1 of class 3 {:8.3f} | '.format(accu_test, f1_0_test, f1_1_test, f1_2_test, f1_3_test))

### Checking relationships between labels for documents

In [ ]:
CATEGORIES = { 1: 'Satire', 2: 'Hoax', 3: 'Propaganda', 4: 'Reliable news' }

reliable, hoax, satire, propaganda = [], [], [], []

all_documents_dataloader = DataLoader(training_dataset, batch_size=1,
                             shuffle=True, collate_fn=collate_batch)

with torch.no_grad():
    for idx, (label, text) in enumerate(all_documents_dataloader):
        model.reset_h_c(text.size()[0])
        output = model(text)
        # Copy the tensor from GPU to CPU in order to convert to numpy array.
        arr = output.cpu().numpy()[0]
        reliable.append(arr[3])
        hoax.append(arr[1])
        satire.append(arr[0])
        propaganda.append(arr[2])

In [ ]:
plt.scatter(reliable, hoax)
plt.xlabel('reliable')
plt.ylabel('hoax')
plt.title('Reliable vs hoax - documents')
plt.show()

plt.scatter(reliable, satire)
plt.xlabel('reliable')
plt.ylabel('satire')
plt.title('Reliable vs satire - documents')
plt.show()

plt.scatter(reliable, propaganda)
plt.title('Reliable vs propaganda - documents')
plt.xlabel('reliable')
plt.ylabel('propaganda')
plt.show()

plt.scatter(hoax, propaganda)
plt.title('Hoax vs propaganda - documents')
plt.xlabel('hoax')
plt.ylabel('propaganda')
plt.show()

plt.scatter(hoax, satire)
plt.title('Hoax vs satire - documents')
plt.xlabel('hoax')
plt.ylabel('satire')
plt.show()

plt.scatter(satire, propaganda)
plt.title('Satire vs propaganda - documents')
plt.xlabel('satire')
plt.ylabel('propaganda')
plt.show()

### Checking relationships between labels for words

In [ ]:
words = vocab.get_itos()

def collate_word(word):
    text_list, offsets = [], [0]
    processed_text = torch.tensor(text_pipeline(word), dtype=torch.int64)
    text_list.append(processed_text)
    offsets.append(processed_text.size(0))
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return text_list.to(device), offsets.to(device)

reliable, hoax, satire, propaganda = [], [], [], []

with torch.no_grad():
    for word in words:
        text, _ = collate_word(list(word))
        output = model(text)
        # Copy the tensor from GPU to CPU in order to convert to numpy array.
        arr = output.cpu().numpy()[0]
        reliable.append(arr[3])
        hoax.append(arr[1])
        satire.append(arr[0])
        propaganda.append(arr[2])

In [ ]:
plt.scatter(reliable, hoax)
plt.xlabel('reliable')
plt.ylabel('hoax')
plt.title('Reliable vs hoax - words')
plt.show()

plt.scatter(reliable, satire)
plt.xlabel('reliable')
plt.ylabel('satire')
plt.title('Reliable vs satire - words')
plt.show()

plt.scatter(reliable, propaganda)
plt.title('Reliable vs propaganda - words')
plt.xlabel('reliable')
plt.ylabel('propaganda')
plt.show()

plt.scatter(hoax, propaganda)
plt.title('Hoax vs propaganda - words')
plt.xlabel('hoax')
plt.ylabel('propaganda')
plt.show()

plt.scatter(hoax, satire)
plt.title('Hoax vs satire - words')
plt.xlabel('hoax')
plt.ylabel('satire')
plt.show()

plt.scatter(satire, propaganda)
plt.title('Satire vs propaganda - words')
plt.xlabel('satire')
plt.ylabel('propaganda')
plt.show()